In [10]:
import numpy as np
import tensorflow as tf
import cv2
import tensorflow_datasets as tfds

In [4]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.NM229Z_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.NM229Z_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [5]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label
scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [6]:
BUFFER_SIZE = 1000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [7]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)
validation_inputs, validation_targets = next(iter(validation_data))
print(validation_inputs.shape)
print(validation_targets.shape)

(6000, 28, 28, 1)
(6000,)


In [8]:
input_size = 784
output_size = 10
hidden_layer_size = 50
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
NUM_EPOCHS = 30
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_data,
          epochs=NUM_EPOCHS,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose=2)

Epoch 1/30
540/540 - 10s - 19ms/step - accuracy: 0.8835 - loss: 0.4072 - val_accuracy: 0.9370 - val_loss: 0.2124
Epoch 2/30
540/540 - 5s - 9ms/step - accuracy: 0.9483 - loss: 0.1782 - val_accuracy: 0.9527 - val_loss: 0.1606
Epoch 3/30
540/540 - 4s - 7ms/step - accuracy: 0.9611 - loss: 0.1337 - val_accuracy: 0.9580 - val_loss: 0.1360
Epoch 4/30
540/540 - 3s - 6ms/step - accuracy: 0.9688 - loss: 0.1082 - val_accuracy: 0.9615 - val_loss: 0.1214
Epoch 5/30
540/540 - 6s - 12ms/step - accuracy: 0.9730 - loss: 0.0918 - val_accuracy: 0.9632 - val_loss: 0.1129
Epoch 6/30
540/540 - 4s - 7ms/step - accuracy: 0.9763 - loss: 0.0798 - val_accuracy: 0.9635 - val_loss: 0.1154
Epoch 7/30
540/540 - 4s - 7ms/step - accuracy: 0.9796 - loss: 0.0689 - val_accuracy: 0.9672 - val_loss: 0.1071
Epoch 8/30
540/540 - 4s - 8ms/step - accuracy: 0.9811 - loss: 0.0618 - val_accuracy: 0.9690 - val_loss: 0.1087
Epoch 9/30
540/540 - 5s - 9ms/step - accuracy: 0.9832 - loss: 0.0553 - val_accuracy: 0.9695 - val_loss: 0.098

In [12]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.9771 - loss: 0.0777
Test loss: 0.08. Test accuracy: 97.71%


In [16]:
img = cv2.imread('/content/cyfra.png', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (28, 28))
img = cv2.bitwise_not(img)
img = img / 255.0
img = img.reshape(1, 28, 28, 1)
prediction = model.predict(img)
predicted_label = np.argmax(prediction)
print(f'Predykcja modelu: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Predykcja modelu: 7
